In [1]:
import pandas as pd
import numpy as np

In [2]:
import pymongo
from pymongo import MongoClient

In [3]:
client = MongoClient('mongodb://localhost:27017')

In [4]:
db=client["dev_comexstat_api2"]

In [5]:
coll_tab01=db["depara_sh4"]
coll_tab13=db["depara_mun"]
coll_tab10=db["depara_pais"]

In [6]:
pipe_tab01 = [{'$project':{'_id':0, 
                           'CO_SH4':1,
                           'NO_SH4_POR':1}}]

pipe_tab10 = [{'$project':{'_id':0,
                           'CO_PAIS': 1,
                           'NO_PAIS': 1 }}]

pipe_tab13 = [{'$project':{'_id':0, 
                           'CO_MUN_GEO': 1,
                           'NO_MUN': 1 }}]

In [7]:
cursor_tab01=coll_tab01.aggregate(pipe_tab01)
cursor_tab10=coll_tab10.aggregate(pipe_tab10)
cursor_tab13=coll_tab13.aggregate(pipe_tab13)

In [8]:
list_tab01=list(cursor_tab01)
list_tab10=list(cursor_tab10)
list_tab13=list(cursor_tab13)

In [9]:
df_tab01=pd.DataFrame(list_tab01)
df_tab10=pd.DataFrame(list_tab10)
df_tab13=pd.DataFrame(list_tab13)

In [10]:
base_imp=r"C:\Users\jetin\Documents\FATEC\2° semestre\api2\Dados\IMP_2023_MUN.csv"
base_exp=r"C:\Users\jetin\Documents\FATEC\2° semestre\api2\Dados\EXP_2023_MUN.csv"

In [11]:
dict_type={'CO_ANO':str,
           'CO_MES':str,
           'CO_MUN':str,
           'CO_PAIS':str,
           'SH4':str }

In [12]:
df_imp=pd.read_csv(base_imp, sep=";", dtype=dict_type)
df_exp=pd.read_csv(base_exp, sep=";", dtype=dict_type)

In [14]:
df_exp

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB
0,2023,07,5201,858,MT,5107958,98971,174398
1,2023,07,8708,158,SP,3450308,22224,345388
2,2023,01,6204,249,SP,3450308,53,7125
3,2023,02,1108,337,RR,1400100,60,174
4,2023,03,1905,580,ES,3205309,160,1083
...,...,...,...,...,...,...,...,...
642043,2023,05,3926,063,RS,4302105,392,2684
642044,2023,06,3001,455,RS,4314100,58,558400
642045,2023,05,2304,365,MG,3170206,9131722,4507056
642046,2023,02,2836,077,RJ,3304557,2,5


#df_imp.describe() - mostra informações numéricas

#df_merge3.groupby(['NO_MUN','CO_PAIS']).count()   - análise

#len(df_merge3.loc[df_merge3['NO_MUN']=='SAO JOSE DOS CAMPOS']['NO_PAIS'].unique())    - análise

#df_merge3.groupby(['NO_MUN']).agg({'NO_PAIS':'nunique'}).reset_index()     - análise

### Base Importação

In [47]:
df_merge1=pd.merge(df_imp, df_tab01, how='left', left_on='SH4', right_on='CO_SH4')
df_merge2=pd.merge(df_merge1, df_tab10, how='left', on='CO_PAIS')
df_merge3=pd.merge(df_merge2, df_tab13, how='left', left_on='CO_MUN', right_on='CO_MUN_GEO')

In [33]:
df_merge3.head(2)

,CO_ANO,CO_MES,KG_LIQUIDO,VL_FOB,CO_SH4,NO_SH4_POR,SG_UF_MUN,CO_MUN_GEO,NO_MUN,NO_PAIS
0,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica"
1,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica"


In [48]:
df_merge3= df_merge3.drop(['SH4', 'CO_MUN','CO_PAIS'], axis=1)

In [49]:
df_merge3.insert(6,'SG_UF_MUN', df_merge3.pop('SG_UF_MUN'))
df_merge3.insert(7,'CO_MUN_GEO', df_merge3.pop('CO_MUN_GEO'))
df_merge3.insert(8,'NO_MUN', df_merge3.pop('NO_MUN'))
df_merge3.insert(9,'NO_PAIS', df_merge3.pop('NO_PAIS'))

In [50]:
df_merge3['tipo'] = 'IMPORTAÇÃO'

In [51]:
df_imp_enr =df_merge3

In [52]:
df_imp_enr.head()

,CO_ANO,CO_MES,KG_LIQUIDO,VL_FOB,CO_SH4,NO_SH4_POR,SG_UF_MUN,CO_MUN_GEO,NO_MUN,NO_PAIS,tipo
0,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO
1,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO
2,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO
3,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO
4,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO


In [ ]:
df_imp_agrup.to_csv('imp_cad_prod.csv')

### Base Exportação

In [38]:
df_merge1=pd.merge(df_exp, df_tab01, how='left', left_on='SH4', right_on='CO_SH4')
df_merge2=pd.merge(df_merge1, df_tab10, how='left', on='CO_PAIS')
df_merge3=pd.merge(df_merge2, df_tab13, how='left', left_on='CO_MUN', right_on='CO_MUN_GEO')

In [39]:
df_merge3= df_merge3.drop(['SH4', 'CO_MUN','CO_PAIS'], axis=1)

In [40]:
df_merge3.insert(6,'SG_UF_MUN', df_merge3.pop('SG_UF_MUN'))
df_merge3.insert(7,'CO_MUN_GEO', df_merge3.pop('CO_MUN_GEO'))
df_merge3.insert(8,'NO_MUN', df_merge3.pop('NO_MUN'))
df_merge3.insert(9,'NO_PAIS', df_merge3.pop('NO_PAIS'))

In [41]:
df_merge3.head(2)

,CO_ANO,CO_MES,KG_LIQUIDO,VL_FOB,CO_SH4,NO_SH4_POR,SG_UF_MUN,CO_MUN_GEO,NO_MUN,NO_PAIS
0,2023,07,98971,174398,5201,"Algod�o, n�o cardado nem penteado",MT,5107958,TANGARA DA SERRA,Vietn�
1,2023,07,22224,345388,8708,Partes e acess�rios dos ve�culos autom�veis da...,SP,3450308,SAO PAULO,Chile


In [42]:
df_merge3['tipo'] = 'EXPORTAÇÃO'

In [45]:
df_exp_enr =df_merge3

In [46]:
df_exp_enr.head()

,CO_ANO,CO_MES,KG_LIQUIDO,VL_FOB,CO_SH4,NO_SH4_POR,SG_UF_MUN,CO_MUN_GEO,NO_MUN,NO_PAIS,tipo
0,2023,07,98971,174398,5201,"Algod�o, n�o cardado nem penteado",MT,5107958,TANGARA DA SERRA,Vietn�,EXPORTAÇÃO
1,2023,07,22224,345388,8708,Partes e acess�rios dos ve�culos autom�veis da...,SP,3450308,SAO PAULO,Chile,EXPORTAÇÃO
2,2023,07,22224,345388,8708,Partes e acess�rios dos ve�culos autom�veis da...,SP,3450308,SAO PAULO,Chile,EXPORTAÇÃO
3,2023,07,22224,345388,8708,Partes e acess�rios dos ve�culos autom�veis da...,SP,3450308,SAO PAULO,Chile,EXPORTAÇÃO
4,2023,07,22224,345388,8708,Partes e acess�rios dos ve�culos autom�veis da...,SP,3450308,SAO PAULO,Chile,EXPORTAÇÃO


In [ ]:
df_exp_agrup.to_csv('exp_cad_prod.csv')

### Junção das Bases Imp e Exp

In [54]:
df_concat_cad_prod=pd.concat([df_imp_enr,df_exp_enr])

In [55]:
df_concat_cad_prod=df_concat_cad_prod.reset_index(drop=True)

In [56]:
df_concat_cad_prod

,CO_ANO,CO_MES,KG_LIQUIDO,VL_FOB,CO_SH4,NO_SH4_POR,SG_UF_MUN,CO_MUN_GEO,NO_MUN,NO_PAIS,tipo
0,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO
1,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO
2,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO
3,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO
4,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO
...,...,...,...,...,...,...,...,...,...,...,...
12867187,2023,02,2,5,2836,Carbonatos; peroxocarbonatos (percarbonatos); ...,RJ,3304557,RIO DE JANEIRO,Bahamas,EXPORTAÇÃO
12867188,2023,02,2,5,2836,Carbonatos; peroxocarbonatos (percarbonatos); ...,RJ,3304557,RIO DE JANEIRO,Bahamas,EXPORTAÇÃO
12867189,2023,04,5,40,6603,"Partes, guarni��es e acess�rios, para os artef...",SC,4218202,TIMBO,Uruguai,EXPORTAÇÃO
12867190,2023,04,5,40,6603,"Partes, guarni��es e acess�rios, para os artef...",SC,4218202,TIMBO,Uruguai,EXPORTAÇÃO


In [57]:
df_concat_cad_prod.duplicated().sum()

11283169

In [58]:
cadeias_produtivas = {'9401' : 'Moveleira','1517' : 'Alimentar','0402' : 'Alimentar','8438' : 'Alimentar','2715' : 'Petroleira',
'8417' : 'Metalúrgica','3907' : 'Petroquímica','2933' : 'Produtos Químicos Orgânicos','3808' : 'Agronegócio',
'2905' : 'Produtos Químicos Orgânicos','3909' : 'Petroleira','2929' : 'Produtos Químicos','2934' : 'Produtos Químicos Orgânicos',
'2931' : 'Produtos Químicos Orgânicos','6306' : 'Têxtil','9506' : 'Cultura e entretenimento','5407' : 'Têxtil',
'8526' : 'Telecomunicações','8452' : 'Têxtil','6815' : 'Minério','5903' : 'Têxtil','7601' : 'Metalúrgica',
'8708' : 'Automotiva','3402' : 'Produtos Químicos Orgânicos','7612' : 'Metalúrgica','2836' : 'Produtos Químicos',
'3105' : 'Agronegócio','3102' : 'Agronegócio','3824' : 'Metalúrgica',
'8411' : 'Aeronáutica','3002' : 'Farmacêutica','8803' : 'Aeronáutica','8807' : 'Aeronáutica','2709' : 'Petroleira',
'2710' : 'Petroleira','1003' : 'Alimentar','1107' : 'Alimentar','2833' : 'Produtos Químicos','8517' : 'Materiais Elétricos',
'8542' : 'Materiais Elétricos','7326' : 'Metalúrgica','8481' : 'Metalúrgica','7007' : 'Vidreira','8544' : 'Materiais Elétricos',
'1905' : 'Alimentar','8607' : 'Automotiva','1806' : 'Alimentar','7112' : 'Minérios','3903' : 'Petroleira',
'4703' : 'Papel e Celulose','4810' : 'Papel e Celulose','7005' : 'Vidreira','8802' : 'Aeronáutica',
'8704' : 'Automotiva','3005' : 'Farmacêutica','0202' : 'Alimentar','0306' : 'Alimentar',
'2202' : 'Alimentar','8703' : 'Automotiva','8410' : 'Metalúrgica','3302' : 'Farmacêutica','3004' : 'Farmacêutica',                     
'8457' : 'Metalúrgica', '2924' : 'Agronegócio', '8407' : 'Automotiva', '3104' : 'Agronegócio', '3304' : 'Farmacêutica',
'9032' : 'Aparelhos Eletrônicos', '8409' : 'Automotiva', '8805' : 'Aeronáutica','8529' : 'Automotiva',
'8439' : 'Papel e Celulose', '8462' : 'Metalúrgica', '9026' : 'Aparelhos Eletrônicos','2924' : 'Produtos Químicos Orgânicos',
'3815' : 'Produtos Químicos', '9031' : 'Aparelhos Eletrônicos', '7010' : 'Farmacéutica','2916' : 'Produtos Químicos Orgânicos',
'7606' : 'Automotiva', '7616' : 'Construção Civil', '8483' : 'Automotiva','8450' : 'Aparelhos Eletrônicos',   
'8415' : 'Aparelhos Eletrônicos', '0405' : 'Alimentar','0801' : 'Alimentar','0802' : 'Alimentar','0813' : 'Alimentar',
'1702' : 'Alimentar', '2008' : 'Alimentar' }

In [60]:
df_concat_cad_prod['cadeia_produtiva'] = df_concat_cad_prod['CO_SH4'].map(cadeias_produtivas)

In [61]:
df_concat_cad_prod

,CO_ANO,CO_MES,KG_LIQUIDO,VL_FOB,CO_SH4,NO_SH4_POR,SG_UF_MUN,CO_MUN_GEO,NO_MUN,NO_PAIS,tipo,cadeia_produtiva
0,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO,NaN
1,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO,NaN
2,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO,NaN
3,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO,NaN
4,2023,03,1844,92086,8413,"Bombas para l�quidos, mesmo com dispositivo me...",SC,4208203,ITAJAI,"Tcheca, Rep�blica",IMPORTAÇÃO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
12867187,2023,02,2,5,2836,Carbonatos; peroxocarbonatos (percarbonatos); ...,RJ,3304557,RIO DE JANEIRO,Bahamas,EXPORTAÇÃO,Produtos Químicos
12867188,2023,02,2,5,2836,Carbonatos; peroxocarbonatos (percarbonatos); ...,RJ,3304557,RIO DE JANEIRO,Bahamas,EXPORTAÇÃO,Produtos Químicos
12867189,2023,04,5,40,6603,"Partes, guarni��es e acess�rios, para os artef...",SC,4218202,TIMBO,Uruguai,EXPORTAÇÃO,NaN
12867190,2023,04,5,40,6603,"Partes, guarni��es e acess�rios, para os artef...",SC,4218202,TIMBO,Uruguai,EXPORTAÇÃO,NaN


In [62]:
df_concat_cad_prod['cadeia_produtiva'] = df_concat_cad_prod['cadeia_produtiva'].fillna('Outras')

In [63]:
df_concat_cad_prod['cadeia_produtiva'].isna().sum()

0

In [64]:
df_concat_cad_prod['cadeia_produtiva'].value_counts()

Outras                         10009206
Automotiva                       579734
Materiais Elétricos              501051
Metalúrgica                      410406
Produtos Químicos Orgânicos      262948
Aparelhos Eletrônicos            206103
Alimentar                        192776
Moveleira                        139377
Farmacêutica                     114644
Agronegócio                       74828
Petroleira                        69441
Petroquímica                      62244
Têxtil                            56760
Cultura e entretenimento          33796
Produtos Químicos                 33724
Papel e Celulose                  28264
Construção Civil                  22422
Aeronáutica                       19844
Minério                           16504
Vidreira                          13728
Telecomunicações                   9249
Farmacéutica                       8694
Minérios                           1449
Name: cadeia_produtiva, dtype: int64

In [65]:
df_concat_cad_prod.to_csv('base_mun_cad_prod.csv')